# 🔍 QualiVault: Validate Transcripts
**Goal:** Use Ollama (local LLM) to detect transcription errors and hallucinations.

1. Loads transcripts from CSV files.
2. Samples segments and sends them to Ollama for validation.
3. Flags potential errors: hallucinations, misheard words, artifacts.
4. Generates validation report with suggestions.

**Prerequisites:**
- Ollama must be installed and running (`ollama serve`)
- Install a model: `ollama pull llama3.1` or `ollama pull jobautomation/OpenEuroLLM-Danish:latest`

In [1]:
# 1. Check Ollama Installation and Available Models
import requests
import json

OLLAMA_URL = "http://localhost:11434"

def check_ollama():
    """Test if Ollama is running and list available models."""
    print("🔍 Checking Ollama installation...\n")
    
    # Test connection
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        
        if response.status_code == 200:
            print("✅ Ollama is running!")
            
            # List available models
            data = response.json()
            models = data.get('models', [])
            
            if models:
                print(f"\n📦 Available models ({len(models)}):\n")
                
                for model in models:
                    name = model.get('name', 'Unknown')
                    size_gb = model.get('size', 0) / (1024**3)
                    
                    # Highlight Danish model
                    if 'danish' in name.lower() or 'openeuro' in name.lower():
                        print(f"   🇩🇰 {name} ({size_gb:.1f} GB) ← Recommended for Danish")
                    elif 'llama3' in name.lower():
                        print(f"   🦙 {name} ({size_gb:.1f} GB) ← Good general model")
                    else:
                        print(f"   • {name} ({size_gb:.1f} GB)")
                
                print("\n💡 Recommendation:")
                danish_models = [m for m in models if 'danish' in m.get('name', '').lower() or 'openeuro' in m.get('name', '').lower()]
                
                if danish_models:
                    print(f"   Use: '{danish_models[0]['name']}' (Danish-optimized)")
                elif any('llama3' in m.get('name', '').lower() for m in models):
                    llama3 = [m for m in models if 'llama3' in m.get('name', '').lower()][0]
                    print(f"   Use: '{llama3['name']}' (general purpose)")
                else:
                    print(f"   Use: '{models[0]['name']}'")
                
                return True, models
            else:
                print("⚠️  Ollama is running but no models are installed!")
                print("\n📥 Install a model:")
                print("   For Danish: ollama pull jobautomation/OpenEuroLLM-Danish:latest")
                print("   General:    ollama pull llama3.1")
                return False, []
        else:
            print(f"❌ Ollama responded with error: {response.status_code}")
            return False, []
            
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama!")
        print("\n🔧 To fix:")
        print("   1. Install Ollama: https://ollama.ai")
        print("   2. Start Ollama: ollama serve")
        print("   3. Pull a model: ollama pull llama3.1")
        return False, []
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False, []

# Run the check
ollama_ok, available_models = check_ollama()

🔍 Checking Ollama installation...

✅ Ollama is running!

📦 Available models (1):

   🦙 llama3.1:latest (4.6 GB) ← Good general model

💡 Recommendation:
   Use: 'llama3.1:latest' (general purpose)


In [2]:
# 2. Load Configuration
%load_ext autoreload
%autoreload 2
import yaml
from pathlib import Path
from qualivault.validation import OllamaValidator, validate_recipe_transcripts

project_root = Path("..").resolve()
config_path = project_root / "config.yml"

with open(config_path) as f:
    config = yaml.safe_load(f)

recipe_path = project_root / "processing_recipe.yaml"
transcripts_dir = (project_root / config['paths']['output_base_folder']).resolve()

print(f"📂 Transcripts: {transcripts_dir}")
print(f"📋 Recipe: {recipe_path}")

📂 Transcripts: D:\legendary2\transcribe
📋 Recipe: C:\dev\qualvalt\projects\Legundary\processing_recipe.yaml


## Configuration

Adjust these settings to control validation:

- **model**: Ollama model to use (choose from available models above)
- **sample_rate**: Fraction of segments to check (0.1 = 10%, 1.0 = 100%)
- **language**: Expected language of transcripts
- **ollama_url**: URL where Ollama is running (default: localhost)

In [3]:
# 3. Validation Settings
# Choose model based on available models above
MODEL = "llama3.1"              # Change to your preferred model
SAMPLE_RATE = 0.05              # Default sample rate
LANGUAGE = "Danish"             # Expected language
MAX_INTERVIEWS = 3              # Limit interviews for iterative tuning (None = all)

# Central place to tweak validation behavior
VALIDATION_PARAMS = {
    "sample_rate": SAMPLE_RATE,     # fraction of segments to check
    "min_text_length": 8,           # skip very short fragments
    "max_segments": 50,             # limit segments per file (None for all)
    "max_interviews": MAX_INTERVIEWS, # limit number of interviews to process
    "ollama_options": {             # passed to Ollama generate options
        "temperature": 0.1,
        "top_p": 0.9
    },
    # Friendly, line-broken prompt a non-coder can edit
    "prompt_template": """
You are validating a {language} interview transcript segment.

Segment: "{text}"
Speaker: {speaker}

Analyze this segment and identify:
1. Hallucinations (repeated phrases, gibberish, random text)
2. Transcription errors (misheard words that don't fit context)
3. Audio artifacts (timestamps, file names, subtitles metadata)
4. Incomplete sentences or fragments

Respond in JSON format:
{{
    "has_issues": true/false,
    "confidence": 0.0-1.0,
    "issues": ["issue1", "issue2"],
    "suggestions": "brief correction suggestion"
}}

Be conservative - only flag clear problems.
"""
}

print(f"🤖 Model: {MODEL}")
print(f"📊 Sample Rate: {VALIDATION_PARAMS['sample_rate'] * 100}%")
print(f"🌍 Language: {LANGUAGE}")
print(f"📄 Max Interviews: {VALIDATION_PARAMS['max_interviews'] or 'All'}")
print(f"⚙️  Max segments/file: {VALIDATION_PARAMS['max_segments']}")

# Quick test
if ollama_ok:
    validator = OllamaValidator(model=MODEL, ollama_url=OLLAMA_URL)
    test_response = validator._query_ollama("Say 'Hello' in one word.", options=VALIDATION_PARAMS.get("ollama_options"))
    
    if test_response:
        print(f"\n✅ Model '{MODEL}' is responding: '{test_response.strip()[:50]}'")
    else:
        print(f"\n❌ Model '{MODEL}' is not responding. Check if it's installed.")
else:
    print("\n⚠️  Skipping test - Ollama not available")

🤖 Model: llama3.1
📊 Sample Rate: 5.0%
🌍 Language: Danish
📄 Max Interviews: 3
⚙️  Max segments/file: 50

✅ Model 'llama3.1' is responding: 'Hiya'


## Validate Transcripts

Run validation on all transcripts in the recipe. This will:
- Sample segments from each transcript
- Check for hallucinations, repeated words, and inconsistencies
- Generate a detailed report with flagged segments

In [4]:
# 4. Run Validation
from datetime import datetime
from pathlib import Path

print(f"🔍 Starting validation at {datetime.now().strftime('%H:%M:%S')}")
print(f"   Model: {MODEL}")
print(f"   Sample Rate: {VALIDATION_PARAMS['sample_rate'] * 100}%")
print(f"   Max Interviews: {VALIDATION_PARAMS['max_interviews'] or 'All'}")
print(f"   Scanning: {transcripts_dir}")
print("=" * 70)

# Find all CSV files to validate (don't depend on recipe status)
csv_files = sorted(transcripts_dir.glob("*.csv"))

# Limit interviews if specified
if VALIDATION_PARAMS.get("max_interviews"):
    csv_files = csv_files[:VALIDATION_PARAMS["max_interviews"]]

if not csv_files:
    print(f"❌ No CSV files found in {transcripts_dir}")
    results = []
else:
    print(f"Found {len(csv_files)} CSV files\n")
    
    results = []
    for i, csv_file in enumerate(csv_files, 1):
        print(f"[{i}/{len(csv_files)}] Validating {csv_file.name}...", end=" ")
        
        validator = OllamaValidator(model=MODEL, ollama_url=OLLAMA_URL)
        report = validator.validate_transcript(
            csv_file,
            sample_rate=VALIDATION_PARAMS["sample_rate"],
            language=LANGUAGE,
            validation_params=VALIDATION_PARAMS
        )
        
        if report:
            results.append(report)
            flagged = len(report.get('flagged_segments', []))
            checked = report.get('segments_checked', 0)
            print(f"✅ {flagged} issues in {checked} segments")
        else:
            print(f"⏭️  Skipped")

print("\n" + "=" * 70)
print(f"✅ Validation complete at {datetime.now().strftime('%H:%M:%S')}")
print(f"   Total transcripts processed: {len(results)}")

🔍 Starting validation at 12:35:56
   Model: llama3.1
   Sample Rate: 5.0%
   Max Interviews: 3
   Scanning: D:\legendary2\transcribe
Found 3 CSV files

[1/3] Validating Interview_1.csv... 

Error querying Ollama: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)


✅ 7 issues in 8 segments
[2/3] Validating Interview_10.csv... ✅ 11 issues in 11 segments
[3/3] Validating Interview_102.csv... ✅ 16 issues in 17 segments

✅ Validation complete at 12:41:21
   Total transcripts processed: 3


## Summary Statistics

Overview of validation results across all transcripts.

In [9]:
# 5. Display Summary
import pandas as pd

# Pull totals from results
total_flagged = sum(r.get('flagged_count', len(r.get('flagged_segments', []))) for r in results)
total_checked = sum(r.get('segments_checked', 0) for r in results)

print(f"📊 VALIDATION SUMMARY")
print(f"=" * 70)
print(f"Files validated: {len(results)}")
print(f"Total segments checked: {total_checked}")
print(f"Total segments flagged: {total_flagged}")
if total_checked > 0:
    print(f"Flag rate: {(total_flagged / total_checked * 100):.1f}%")
print(f"=" * 70)

# Per-file breakdown
df = pd.DataFrame([
    {
        'File': Path(r.get('csv_file', 'unknown')).stem,
        'Checked': r.get('segments_checked', 0),
        'Flagged': r.get('flagged_count', len(r.get('flagged_segments', []))),
        'Rate': f"{(r.get('flagged_count', len(r.get('flagged_segments', []))) / max(r.get('segments_checked', 1), 1) * 100):.1f}%" 
               if r.get('segments_checked', 0) > 0 else "N/A"
    }
    for r in results
])

print("\n📁 Per-File Results:")
print(df.to_string(index=False))

📊 VALIDATION SUMMARY
Files validated: 3
Total segments checked: 36
Total segments flagged: 34
Flag rate: 94.4%

📁 Per-File Results:
         File  Checked  Flagged   Rate
  Interview_1        8        7  87.5%
 Interview_10       11       11 100.0%
Interview_102       17       16  94.1%


## Common Transcription Errors

Domain-specific errors and suspicious patterns found across all transcripts.
Use this list to create auto-correction rules.


In [11]:
# 6a. Extract Common Errors (Systemic Issues - by TYPE)
from qualivault.validation import extract_common_errors

common_errors = extract_common_errors(results, top_n=15)

if common_errors:
    print("⚠️  MOST COMMON ERROR TYPES")
    print("=" * 70)
    print("Error Type                           | Count | %")
    print("-" * 70)
    
    # Calculate total for percentages
    total_errors = sum(e['count'] for e in common_errors)
    
    for i, error_info in enumerate(common_errors, 1):
        error_type = error_info['error_type']
        count = error_info['count']
        percentage = (count / total_errors * 100) if total_errors > 0 else 0
        
        # Truncate long error types
        display_type = error_type[:30] if len(error_type) <= 30 else error_type[:27] + "..."
        
        print(f"{i:2d}. {display_type:<30} | {count:>4d} | {percentage:>5.1f}%")
    
    print("\n💡 Use these patterns to create auto-correction rules or improve the Ollama prompt")
else:
    print("✅ No common errors identified!")

⚠️  MOST COMMON ERROR TYPES
Error Type                           | Count | %
----------------------------------------------------------------------
 1. Transcription error            |   28 |  42.4%
 2. Incomplete sentence or frag... |   20 |  30.3%
 3. {'type'                        |    9 |  13.6%
 4. Incomplete sentence            |    4 |   6.1%
 5. Incomplete sentence/fragment   |    3 |   4.5%
 6. Hallucinations                 |    1 |   1.5%
 7. Transcription errors           |    1 |   1.5%

💡 Use these patterns to create auto-correction rules or improve the Ollama prompt


In [12]:
# 6a. Extract Common Errors (Systemic Issues)
from qualivault.validation import extract_common_errors

common_errors = extract_common_errors(results, top_n=15)

if common_errors:
    print("⚠️  MOST COMMON TRANSCRIPTION ERRORS")
    print("=" * 70)
    
    # Create DataFrame for nice display
    error_df = pd.DataFrame(common_errors)
    
    # Add rank column
    error_df.insert(0, 'Rank', range(1, len(error_df) + 1))
    
    print(error_df.to_string(index=False))
    
    print("\n💡 Use these to create auto-correction rules or adjust Ollama prompt")
else:
    print("✅ No common errors identified!")

⚠️  MOST COMMON TRANSCRIPTION ERRORS
 Rank                      error_type  count
    1             Transcription error     28
    2 Incomplete sentence or fragment     20
    3                         {'type'      9
    4             Incomplete sentence      4
    5    Incomplete sentence/fragment      3
    6                  Hallucinations      1
    7            Transcription errors      1

💡 Use these to create auto-correction rules or adjust Ollama prompt


## Export Report

Save validation results to JSON for later review or processing.

In [13]:
# 7. Export Results
import json

report_path = project_root / "validation_report.json"

export_data = {
    'validation_date': datetime.now().isoformat(),
    'model': MODEL,
    'sample_rate': SAMPLE_RATE,
    'language': LANGUAGE,
    'summary': {
        'files_validated': len(results),
        'total_segments_checked': total_checked,
        'total_segments_flagged': total_flagged,
        'flag_rate': f"{(total_flagged / total_checked * 100):.1f}%" if total_checked > 0 else "0%"
    },
    'results': results
}

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print(f"💾 Report saved to: {report_path}")
print(f"   Size: {report_path.stat().st_size / 1024:.1f} KB")

💾 Report saved to: C:\dev\qualvalt\projects\Legundary\validation_report.json
   Size: 27.6 KB
